Topic model with Ollama (to generate human readable topic labels)

We first test with LDA topic model  
Then test with BERTopic, and CTM

Tutorial: https://python.langchain.com/docs/integrations/llms/ollama

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama

Init Ollama object

langchain api: https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html#  
ollama api: https://github.com/jmorganca/ollama/blob/main/docs/api.md

basically the api is identical, just pass the params in the init()

In [2]:
llm = Ollama(model="llama2")        # assuming the port is 11434

In [3]:
llm.invoke("Tell me about history of AI")

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nArtificial intelligence (AI) has a rich and diverse history that spans several decades. Here is a brief overview:\n\n1. 1950s-1960s: The Dartmouth Conference and the Birth of AI - The field of AI was founded in 1956 at a conference held at Dartmouth College in Hanover, New Hampshire. Attendees included computer scientists, mathematicians, and cognitive scientists who were interested in exploring the possibilities of creating machines that could simulate human intelligence.\n2. 1951: The Term "Artificial Intelligence" - John McCarthy, a computer scientist and cognitive scientist, coined the term "artificial intelligence" at the Dartmouth Conference. Prior to this, the field was known as "computer science" or "cybernetics."\n3. 1956: The First AI Program - Alan Turing, a British mathematician and computer scientist, developed the first AI program, called the "Turing Test," which was designed to test a machine\'s ability to mimic human conversation.\n4. 1960s: Rule-Based Expert Systems

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me about history of {topic}")
chain = prompt | llm

In [10]:
prompt

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='Tell me about history of {topic}'))])

In [7]:
chain.invoke({"topic":"Blockchain"})

'\nSure, I\'d be happy to explain the history of blockchain!\n\nBlockchain technology has its roots in the early 1990s when computer scientist David Chaum began developing a concept he called "Blind Signature" technology. This technology allowed for secure digital signatures without relying on trusted third parties.\n\nIn the late 1990s and early 2000s, cryptographers like Nick Szabo and Hal Finney began exploring the potential of blockchain technology beyond just digital signatures. They envisioned a decentralized system where transactions could be verified and recorded without the need for intermediaries like banks or governments.\n\nThe modern blockchain concept was born in 2008 with the release of the Bitcoin whitepaper by an anonymous individual or group using the pseudonym Satoshi Nakamoto. The whitepaper proposed a decentralized digital currency that could be secured through cryptography and verified through a network of computers rather than relying on a central authority.\n\nS

How to interact with langchain

Quick Start  
https://python.langchain.com/docs/modules/model_io/quick_start

Writing prompt with ChatPromptTemplate()  
https://python.langchain.com/docs/modules/model_io/prompts/quick_start

https://python.langchain.com/docs/modules/model_io/prompts/composition

The topic keywords and top reviews are taken from the script ctm_demo_gridsearch.ipynb

In [47]:
# system_message = "You are a moderator of an online gaming forum that allows players to post reviews about different games."
system_message = "You are a player of the game who is reading the reviews of the game."
human_template = \
"Create a name for a topic given the topic's keywords and some most representative reviews of the topic. The name of the game is 'Terraria'. The top keywords of the topic is: \'\'\'{topic_keywords}\'\'\'. The most representative reviews of the topic are: \'\'\'{topic_reviews}\'\'\'. Output a description less than 5 words for the topic. Do not output other text."

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_template)
])

topic_top_keywords = [
    'play',
    'hour',
    'buy',
    'time',
    'pc',
    'xbox',
    'first',
    'love',
    'life',
    'would'
]

topic_top_reviews = [
    'Own this on xbox? Pshh, you do not know what you are missing out on. I played the heck of terraria on xbox 360 and I thought I was done playing this game. The PC version is way better and more smooth then xbox 360. 10/10 would play again. One last thing, do not buy the 4-pack unless you 100% know that the friends you bought it for would play this game. I gave it away to 3 of my friends. One ended up playing a good bit of time. the second played for 10 minutes, and the third ended up removing me after I gifted it away. #fail',
    "Well, let me tell you about this game. My son LOVES it. I got it for him around the winter sale as a gift. He has been playing it non-stop for a long time. He has been bugging me to get it and play it to, but it just didn't look entertaining. BOY, was I wrong. I just bought it during the last summer sale (a few weeks ago) and been playing it ALOT. It is worth it and a GREAT break from daily life. ENJOY! ",
    'I love this game, i play it everyday and every night when i get a chance. I was first shown this game on the XboX with my brother playing it at his house, i seen it and first thought that was the dumbest thing ive ever A side scrolling minecraft? So he begged me to try it and after the first 10 mins i was HOOKED! Hooked i tell you! i couldnt take my eyes off the game for a second. I played the game straight for 29 hours none stop except to eat and use the bathroom and occasional xbox freezes from overuse. Man this game is awesome. this game is alot funner if you have friends that play with you. Besides the mutiplayer part i been playing well over 300 hours and still going strong. I HIGHLY suggest this game to ANYONE that loves to play minecraft. ive already gotten 20 of my other friends to buy and play this game and they all love it. Sadly most of their computers broke and i no longer have friends to play mutiplayer with anymore. I need friends to play with shoot me a game invite. PLAY PLAY PLAY you will not be dissapointed! *sorry for bad spelling*',
    ":Heres A True Story: I remember when this game was in stores and everyone was playing it, buying it and loving this game until, 'Minecraft' came out and I never heard anyone talking about Terraria again :,( 3 Years later (2014) I bought this game for Ps3 and I loved it and playing it for ages made me love it even more but the Ps3 Version was glitched. 5 Months later (Still 2014) I bought Terraria for the Pc and I played it more than my Ps3 Version and WOW IT'S AMAZING!!!! 1st January - May (2015) Oh, wow Terraria is being updated in June, time to start playing again! 50 Years later (2060) I lived a good life (I died)",
    'I first got Terraria ios for my kindle 2 years ago and i loved i started playing it again and realized how limited it is so i bought it on steam and i knew what to do right off the bat and i had a blast playing it can wait to get back to it and play some more Terraria! Buy This Game Love it! out of 5',
    # 'purchases game on xbox plays for days with his friends is complete and total noob stops splaying because friends start hacking/cheating months Gets new compute gets steam account buys terraria gets friend to buy it plays for days with friends friends start cheating stops playing months later comes out plays for days with friends surprisingly friends dont cheat and we all have a jolly good time! oh yeah and its a good game too, not a noob anymore , i beat the moonlred like 20 no-noones proud of me? :( ',
    # 'DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE DONT LET ME GET INTO MY ZONE',
    # 'This game is People that berely go on the computer play for 10-70 hours Normal People that go on computers play 70-150 hours People that go on the computer frequently plays 150-500 hours Totally addicted players play for 500-2500 hours People that think they are in terraria 2500- ∞ What will happen to us in a few ',
    # "Didn't want to buy this game at first, but my friend Sara told me to buy as it was on sale as my outher friends bought this game too. At first 4 of us played for the first 2 days and after that i played alone as all of them don't want to play. 11/10 And btw fu*k you Jon, waste my money! Awesome game btw :D",
    # "Bee Movie Script According to all known laws of aviation, there is no way a bee should be able to fly. Its wings are too small to get its fat little body off the ground. The bee, of course, flies anyway because bees don't care what humans think is impossible. Yellow, black. Yellow, black. Yellow, black. Yellow, black. Ooh, black and yellow! Let's shake it up a little. Barry! Breakfast is ready! Ooming! Hang on a second. Hello? - Barry? - Adam? - Oan you believe this is happening? - I can't. I'll pick you up. Looking sharp. Use the stairs. Your father paid good money for those. Sorry. I'm excited. Here's the graduate. We're very proud of you, son. A perfect report card, all B's. Very proud. Ma! I got a thing going here. - You got lint on your fuzz. - Ow! That's me! - Wave to us! We'll be in row 118,000. - Bye! Barry, I told you, stop flying in the house! - Hey, Adam. - Hey, Barry. - Is that fuzz gel? - A little. Special day, graduation. Never thought I'd make it. Three days grade school, three days high school. Those were awkward. Three days college. I'm glad I took a day and hitchhiked around the hive. You did come back different. - Hi, Barry. - Artie, growing a mustache? Looks good. - Hear about Frankie? - Yeah. - You going to the funeral? - No, I'm not going. Everybody knows, sting someone, you die. Don't waste it on a squirrel. Such a hothead. I guess he could have just gotten out of the way. I love this incorporating an amusement park into our day. That's why we don't need vacations. Boy, quite a bit of under the circumstances. - Well, Adam, today we are men. - We are! - Bee-men. - Amen! Hallelujah! Students, faculty, distinguished bees, please welcome Dean Buzzwell. Welcome, New Hive Oity graduating class That concludes our ceremonies. And begins your career at Honex Industries! Will we pick ourjob today? I heard it's just orientation. Heads up! Here we go. Keep your hands and antennas inside the tram at all times. - Wonder what it'll be like? - A little scary. Welcome to Honex, a division of Honesco and a part of the Hexagon Group. This is it! Wow. Wow. We know that you, as a bee, have worked your whole life to get to the point where you can work for your whole life. Honey begins when our valiant Pollen Jocks bring the nectar to the hive. Our top-secret formula is automatically color-corrected, scent-adjusted and bubble-contoured into this soothing sweet syrup with its distinctive golden glow you know Honey! - That girl was hot. - She's my cousin! - She is? - Yes, we're all cousins. - Right. You're right. - At Honex, we constantly strive to improve every aspect of bee existence. These bees are stress-testing a new helmet technology. - What do you think he makes? - Not enough. Here we have our latest advancement, the Krelman. - What does that do? - Oatches that little strand of honey that hangs after you pour it. Saves us millions. Oan anyone work on the Krelman? Of course. Most bee jobs are small ones. But bees know that every small job, if it's done well, means a lot. But choose carefully because you'll stay in the job you pick for the rest of your life. The same job the rest of your life? I didn't know that. What's the difference? You'll be happy to know that bees, as a species, haven't had one day off in 27 million years. So you'll just work us to death? We'll sure try. Wow! That blew my mind! 'What's the difference?' How can you say that? One job forever? That's an insane choice to have to make. I'm relieved. Now we only have to make one decision in life. But, Adam, how could they never have told us that? Why would you question anything? We're bees. We're the most perfectly functioning society on Earth. You ever think maybe things work a little too well here? Like what? Give me one example. I don't know. But you know what I'm talking about. Please clear the gate. Royal Nectar Force on approach. Wait a second. Oheck it out. - Hey, those are Pollen Jocks! - Wow. I've never seen them this close. They know what it's like outside the hive. Yeah, but some don't come back. - Hey, Jocks! - Hi, Jocks! You guys did great! You're monsters! You're sky freaks! I love it! I love it! - I wonder where they were. - I don't know. Their day's not planned. Outside the hive, flying who knows where, doing who knows what. You can'tjust decide to be a Pollen Jock. You have to be bred for that. Right. Look. That's more pollen than you and I will see in a lifetime. It's just a status symbol. Bees make too much of it. Perhaps. Unless you're wearing it and the ladies see you wearing it. Those ladies? Aren't they our cousins too? Distant. Distant. Look at these two. - Oouple of Hive Harrys. - Let's have fun with them. It must be dangerous being a Pollen Jock. Yeah. Once a bear pinned me against a mushroom! He had a paw on my throat, and with the other, he was slapping me! - Oh, my! - I never thought I'd knock him out. What were you doing during this? Trying to alert the authorities. I can autograph that. A little gusty out there today, wasn't it, comrades? Yeah. Gusty. We're hitting a sunflower patch six miles from here tomorrow. - Six miles, huh? - Barry! A puddle jump for us, but maybe you're not up for it. - Maybe I am. - You are not! We're going 0900 at J-Gate. What do you think, buzzy-boy? Are you bee enough? I might be. It all depends on what 0900 means. Hey, Honex! Dad, you surprised me. You decide what you're interested in? - Well, there's a lot of choices. - But you only get one. Do you ever get bored doing the same job every day? Son, let me tell you about stirring. You grab that stick, and you just move it around, and you stir it around. You get yourself into a rhythm. It's a beautiful thing. You know, Dad, the more I think about it, maybe the honey field just isn't right for me. You were thinking of what, making balloon animals? That's a bad job for a guy with a stinger. Janet, your son's not sure he wants to go into honey! - Barry, you are so funny sometimes. - I'm not trying to be funny. You're not funny! You're going into honey. Our son, the stirrer! - You're gonna be a stirrer? - No one's listening to me! Wait till you see the sticks I have. I could say anything right now. I'm gonna get an ant tattoo! Let's open some honey and celebrate! Maybe I'll pierce my thorax. Shave my antennae. Shack up with a grasshopper. Get a gold tooth and call everybody 'dawg'! I'm so proud. - We're starting work today! - Today's the day. Oome on! All the good jobs will be gone. Yeah, right. Pollen counting, stunt bee, pouring, stirrer, front desk, hair - Is it still available? - Hang on. Two left! One of them's yours! Oongratulations! Step to the side. - What'd you get? - Picking crud out. Stellar! Wow! Oouple of newbies? Yes, sir! Our first day! We are ready! Make your choice. - You want to go first? - No, you go. Oh, my. What's available? Restroom attendant's open, not for the reason you think. - Any chance of getting the Krelman? - Sure, you're on. I'm sorry, the Krelman just closed out. Wax monkey's always open. The Krelman opened up again. What happened? A bee died. Makes an opening. See? He's dead. Another dead one. Deady. Deadified. Two more dead. Dead from the neck up. Dead from the neck down. That's life! Oh, this is so har",
]

chain = chat_prompt | llm 


In [48]:
chain.invoke({
    "topic_keywords": topic_top_keywords,
    "topic_reviews": topic_top_reviews
})

'Exciting Adventure Game'

In [53]:
topic_top_keywords  = [
    'content',
    'update',
    'game',
    'hour',
    'new',
    'one',
    'time',
    'developer',
    'play',
    'still'
]

topic_top_reviews = [
"LEGIT THE BEST GAME EVER BETTER THAN MINECRAFT, SO MANY THINGS TO DO!!!!!!! BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!BEST GAME EVER!!!",
"Just realized I've had this game for years and never reviewed it. Which is just horrible of me. Of all the games in my steam library this deserves a review. I've enjoyed this game a lot. I've also enjoyed reading the games development storie. The dev stopped development on this game years ago. Was 'encouraged' by his girlfriend to update it. That encouragement must have been awesome. He released not only one more major update for it, but nominated a team of developers to continue work on it for long after that. They are still updating it. Just this month (November of 2016) they released another update that in many games would have been a entire paid DLC but in this game is just par for the course of another free update. After this game was released it has seen many changes and updates. Adding content. Not nerfiing content, but adding more content. Even takign the time to fix bugs a long the way. They game is great. The amount of content I have gotten for free since I have bought this game is amazing though. Most AAA developers might have released a patch for it after release. Maybe a few pieces of paid DLC. The developers on this have released multiple patches a year for years after it was released. Most those patches equivalent to a paid DLC from a AAA title. Not sayign this is the only way to dev a game. Just saying it is a damn nice one to see. Thanks.",
"Simply one of the best indie exploration-sandbox game of all time? I'm writing this review after the HUGE update that brought so much to the game than, after hundreds of hours already spent on this game, I still felt I was up for another nth playthrough, this time in goddam painful 'Expert mode' (but I'm enjoying it!) Seriously, which developer cherish more his baby than Terraria's one? The game is out since 2011, and yet 4 years after, you just happen to get a ton of new content for free for the thord time, for a game that has already been bought by so many people here. There is no profit involved here, just one man's dedication for his game and his community. If all developers could be as much Terraria is simply one of the best gaming experience I ever had. - It's retro stylish, without being ugly nor botched. - Musics are great and stick to the head as you could expect on old videogames. - There are a lot of things to do to really beat the game. - Exploring is both entertaining and rewarding. - Classic combat style and yet so much diversified thanks to a ton of different weapons. - I never felt like I was grinding for something. - You can build so much things, with so much - It's just totally amazing to watch all the crazy/beautiful things the community already made with this game. - Multiplayer is such a wonderful experience here, and it's even better since they made it easier to organize. What else could we ask for? I would buy this again.",
"If these side scroller retro crafting type games appeal to you at all, this is one of the best. Absolutely recommended. Tons of content for the price. Probably the best game I've ever bought for this price! And they just keep adding more content!! Edit/update: Every time I scroll past this game in my Steam library, it makes me smile. I can't wait for a sequel/expansion/major update. What a game!",
"Terraria is a truly amazing game experience, over the years countless updates have been added to the game enhancing it above and beyond what was already a great game. Terraria is a great game if you enjoy action adventure, with the potential to also build your hearts desire (in a 2D space). If you enjoy these kinds of games and have not played Terraria I highly recommend it. Great value for price and still to this day the developer over at Redigit have continued to put out countless updates adding new content for free.",
# "A super fun and addictive Sandbox Crafting 2D Side Scrolling Retro RPG. The amount of content and work put in to this little indie game is astounding. Literally as many hours or more of gameplay as The Witcher 3 right up until endgame, not to mention the insane replayability all thanks to the major free content updates the devs provide us every few months. One of the best price/content ratio games on steam right now, i got it for around $4-5 CND when it went on sale one time and it's one of the best choices i made on steam. Trust me, this game is well worth your hard earned money if you are a fan of the genre. Pick it up, you won't be disappointed.",
# "Terraria is currently the game I've invested the longest amount of time into. When I first booted up the game in one of its early versions, I was one of the players that easily classified Terraria as 'Minecraft, minus one Of course, back then I would not have imagined the plethora of free updates and content Terraria would become throughout its development cycle. Re-Logic, Terraria's developers, have given the gaming community one of the most content-rich, sandbox-exploration games. Even after they began other projects, Re-Logic continues to surprise Terraria communities with new content years after Terraria's release. At the time of writing this review, the game version of Terraria has reached If there was ever an announcement for yet another massive update in the form of I wouldn't be surprised, but excited to start from scratch to enjoy the new additions.",
# "Fantastic game, every time I play it the developer has added new stuff (all for free, not microtransactions). I'm completely amazed by the amount of support the developer has given this game. One of the best games I've ever played, it's a 2D sidescrolling open world adventure game. It has hours on hours of content for a fantastically low price",
# "I've played through this game multiple times with multiple groups of varying size. It's fun even when playing alone, but with a group of people this is pretty much the best game involving crafting that you can pick up. It has tons of content, the combat system is much better than most games of this type, it runs on pretty much anything, it's regularly on sale, the soundtrack is great, and it still gets gigantic free content updates years after release. I own it on Steamand DRM-free on GoG and the Humble Store and I've bought multiple copies for friends. over the years. If you've spent more than 10$ on computer games in your life and don't have this one in your library, you've made a mistake. The only reason not to get this game if you have the chance is if you really despise the genre or have nobody to play it with. The developers deserve praise perhaps more than any other developer for their continued free support and the many community suggestions they've implemented. TL;DR: This game is absolutely amazing, and well worth the asking price. If you don't pick it up full-price, get it on one of the many sales. A blast to play with a group of people.",
# "Much more than a 2D Minecraft, Terraria blends one of the deepest crafting systems of all time, a charming graphical style, Metroidvania elements and epic boss fights together into a game that's more than the sum of its parts. And best of all, the game's only continued to improve, greatly deepening in new features, graphics, weapons, bosses, and even adding entirely new ways to play the game, years after the original release. Almost anyone else would sell the content they give out for free as expansion packs. Not this team. Terraria continues to grow, with the update approaching. While much smaller in scope than the expansion pack-sized it shows the continued dedication that exists to improving an already incredible game. Must buy."
]

In [54]:
chain.invoke({
    "topic_keywords": topic_top_keywords,
    "topic_reviews": topic_top_reviews
})

'Exciting game with endless content'

In [59]:
human_template2 = \
"Create a name for a topic given the topic's keywords. The name of the game is 'Terraria'. The top keywords of the topic is: \'\'\'{topic_keywords}\'\'\'. Output a description less than 5 words for the topic. Do not output other text."

chat_prompt2 = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_template2)
])

chain2 = chat_prompt2 | llm

In [60]:
chain2.invoke({
    "topic_keywords": topic_top_keywords,
    # "topic_reviews": topic_top_reviews
})

'\nNew content in Terraria!'